In [2]:
import requests
import pandas as pd
import psycopg2
import json

In [3]:
url = "https://api.rentcast.io/v1/properties"
headers = {
    "Accept": "application/json",
    "X-Api-Key": "b07dd4766afe474c858ede113ec62d6a"
}

cities = [
    {"city": "Austin", "state": "TX"},
    {"city": "Seattle", "state": "WA"},
    {"city": "Phoenix", "state": "AZ"},
    {"city": "Miami", "state": "FL"}
]

property_limit = 50
all_properties = [] 

for loc in cities:
    print(f"\nFetching properties for {loc['city']}, {loc['state']}...")
    response = requests.get(url, headers=headers, params={**loc, "limit": property_limit})

    if response.status_code == 200:
        data = response.json()
        all_properties.extend(data)  
        for property in data:
            print(property)
    else:
        print(f"API Error {response.status_code}: {response.text}")



Fetching properties for Austin, TX...
{'id': '7011-W-Parmer-Ln,-Austin,-TX-78729', 'formattedAddress': '7011 W Parmer Ln, Austin, TX 78729', 'addressLine1': '7011 W Parmer Ln', 'addressLine2': None, 'city': 'Austin', 'state': 'TX', 'zipCode': '78729', 'county': 'Williamson', 'latitude': 30.450651, 'longitude': -97.749853, 'propertyType': 'Apartment', 'yearBuilt': 2000}
{'id': '5200-N-Lamar-Blvd,-Austin,-TX-78751', 'formattedAddress': '5200 N Lamar Blvd, Austin, TX 78751', 'addressLine1': '5200 N Lamar Blvd', 'addressLine2': None, 'city': 'Austin', 'state': 'TX', 'zipCode': '78751', 'county': 'Travis', 'latitude': 30.320159, 'longitude': -97.731593, 'propertyType': 'Single Family', 'bedrooms': 1, 'bathrooms': 1, 'squareFootage': 505, 'yearBuilt': 1984}
{'id': '1301-W-5th-St,-Austin,-TX-78703', 'formattedAddress': '1301 W 5th St, Austin, TX 78703', 'addressLine1': '1301 W 5th St', 'addressLine2': None, 'city': 'Austin', 'state': 'TX', 'zipCode': '78703', 'county': 'Travis', 'latitude': 

In [4]:
file_name = 'real_estate.json'

with open(file_name, 'w') as file:
    json.dump(all_properties, file, indent=4)

In [5]:
real_estate_df = pd.read_json('real_estate.json')
real_estate_df.head()

,id,formattedAddress,addressLine1,addressLine2,city,state,zipCode,county,latitude,longitude,...,features,taxAssessments,propertyTaxes,lotSize,assessorID,legalDescription,subdivision,owner,ownerOccupied,hoa
0,"7011-W-Parmer-Ln,-Austin,-TX-78729","7011 W Parmer Ln, Austin, TX 78729",7011 W Parmer Ln,None,Austin,TX,78729,Williamson,30.450651,-97.749853,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"5200-N-Lamar-Blvd,-Austin,-TX-78751","5200 N Lamar Blvd, Austin, TX 78751",5200 N Lamar Blvd,None,Austin,TX,78751,Travis,30.320159,-97.731593,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"1301-W-5th-St,-Austin,-TX-78703","1301 W 5th St, Austin, TX 78703",1301 W 5th St,None,Austin,TX,78703,Travis,30.271746,-97.759877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"1044-Camino-Ln,-Costa,-Austin,-TX-78752","1044 Camino Ln, Costa, Austin, TX 78752",1044 Camino Ln,Costa,Austin,TX,78752,Travis,30.325260,-97.699270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"800-W-38th-St,-Austin,-TX-78705","800 W 38th St, Austin, TX 78705",800 W 38th St,None,Austin,TX,78705,Travis,30.304715,-97.741423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
real_estate_df.columns

Index(['id', 'formattedAddress', 'addressLine1', 'addressLine2', 'city',
       'state', 'zipCode', 'county', 'latitude', 'longitude', 'propertyType',
       'yearBuilt', 'bedrooms', 'bathrooms', 'squareFootage', 'features',
       'taxAssessments', 'propertyTaxes', 'lotSize', 'assessorID',
       'legalDescription', 'subdivision', 'owner', 'ownerOccupied', 'hoa'],
      dtype='object')

In [7]:
real_estate_df['yearBuilt'] = real_estate_df['yearBuilt'].fillna(1900)
real_estate_df['ownerOccupied'] = real_estate_df['ownerOccupied'].fillna(False)

key_cols = ['propertyType', 'owner', 'formattedAddress']
non_keys = [col for col in real_estate_df.columns if col not in key_cols]
real_estate_df[non_keys] = real_estate_df[non_keys].fillna('Unknown')


In [8]:
real_estate_df['taxAssessments'] = real_estate_df['taxAssessments'].astype(str)
real_estate_df['propertyTaxes'] = real_estate_df['propertyTaxes'].astype(str)
real_estate_df['assessorID'] = real_estate_df['assessorID'].astype(str)
real_estate_df['legalDescription'] = real_estate_df['legalDescription'].astype(str)
real_estate_df['subdivision'] = real_estate_df['subdivision'].astype(str)
real_estate_df['hoa'] = real_estate_df['hoa'].astype(str)
real_estate_df['formattedAddress'] = real_estate_df['formattedAddress'].astype(str)
real_estate_df['owner'] = real_estate_df['owner'].astype(str)
real_estate_df['propertyType'] = real_estate_df['propertyType'].astype(str)
real_estate_df['addressLine1'] = real_estate_df['addressLine1'].astype(str)
real_estate_df['county'] = real_estate_df['county'].astype(str)
real_estate_df['city'] = real_estate_df['city'].astype(str)
real_estate_df['state'] = real_estate_df['state'].astype(str)

In [9]:
# Creating the dimension tables

location_dim = real_estate_df[['formattedAddress', 'addressLine1', 'addressLine2', 
                               'county', 'zipCode', 'city', 'state',
                               'latitude', 'longitude']].copy().drop_duplicates().reset_index(drop=True)

location_dim.index.name = 'location_id' # Giving the index column a name.

location_dim = location_dim.reset_index() # Converting the index into a proper/recognized column

In [19]:
cols_to_check = [
    'propertyType', 'bedrooms', 'bathrooms', 'squareFootage',
    'features', 'lotSize', 'hoa'
]

real_estate_df[cols_to_check] = real_estate_df[cols_to_check].applymap(
    lambda x: json.dumps(x) if isinstance(x, (dict, list)) else str(x)
)

real_estate_df['features'] = real_estate_df['features'].replace(
    ['N/A', 'nan', 'None', '', 'null', 'Unknown'], 'None'
)


features_dim = real_estate_df[cols_to_check].drop_duplicates().reset_index(drop=True)
features_dim.index.name = 'features_id'
features_dim = features_dim.reset_index()

features_dim.replace(
    ['N/A', 'NA', '', 'null', 'Unknown', 'None'], None, inplace=True
)

numeric_cols = ['bedrooms', 'bathrooms', 'squareFootage', 'lotSize']
for col in numeric_cols:
    features_dim[col] = pd.to_numeric(features_dim[col], errors='coerce')

features_dim = features_dim.query(
    'squareFootage < 1000000 and bedrooms < 100 and bathrooms < 100 and lotSize < 1000000'
).copy()

features_dim['features'] = features_dim['features'].apply(
    lambda x: json.dumps(x) if isinstance(x, (dict, list)) else x
)

real_estate_df['bedrooms'] = pd.to_numeric(real_estate_df['bedrooms'], errors='coerce')
real_estate_df['bathrooms'] = pd.to_numeric(real_estate_df['bathrooms'], errors='coerce')
real_estate_df['squareFootage'] = pd.to_numeric(real_estate_df['squareFootage'], errors='coerce')
real_estate_df['lotSize'] = pd.to_numeric(real_estate_df['lotSize'], errors='coerce')

features_dim['bedrooms'] = pd.to_numeric(features_dim['bedrooms'], errors='coerce')
features_dim['bathrooms'] = pd.to_numeric(features_dim['bathrooms'], errors='coerce')
features_dim['squareFootage'] = pd.to_numeric(features_dim['squareFootage'], errors='coerce')
features_dim['lotSize'] = pd.to_numeric(features_dim['lotSize'], errors='coerce')


print(features_dim.dtypes)
print(features_dim.head())


features_id        int64
propertyType      object
bedrooms         float64
bathrooms        float64
squareFootage    float64
features          object
lotSize          float64
hoa               object
dtype: object
    features_id   propertyType  bedrooms  bathrooms  squareFootage features  \
8             8      apartment       2.0        2.0          704.0     None   
58           58  single family       3.0        3.0         1923.0     None   
76           76      apartment       2.0        2.0        48053.0     None   

    lotSize   hoa  
8   43560.0  None  
58   5580.0  None  
76  51806.0  None  


C:\Users\luk\AppData\Local\Temp\ipykernel_20384\834059164.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  real_estate_df[cols_to_check] = real_estate_df[cols_to_check].applymap(


In [11]:
owner_dim = real_estate_df[['owner']]

owner_dim.index.name = 'owner_id'

owner_dim = owner_dim.reset_index()

In [12]:
def clean_key_col(df, col):
    df[col] = df[col].astype(str).str.strip().str.lower()
    return df

cols_to_clean = ['propertyType', 'owner', 'formattedAddress']

for col in cols_to_clean:
    real_estate_df = clean_key_col(real_estate_df, col)
    if col in features_dim: features_dim = clean_key_col(features_dim, col)
    if col in owner_dim: owner_dim = clean_key_col(owner_dim, col)
    if col in location_dim: location_dim = clean_key_col(location_dim, col)

In [20]:
property_fact = real_estate_df.merge(location_dim, on=['formattedAddress', 'addressLine1', 'addressLine2', 
                               'county', 'zipCode', 'city', 'state',
                               'latitude', 'longitude'], how='left') \
                               .merge(features_dim, on=['propertyType', 'bedrooms', 'bathrooms', 'squareFootage', 
                               'features', 'lotSize'], how='left') \
                               .merge(owner_dim, on=['owner'], how='left') \
                               [['id', 'location_id', 'features_id', 'owner_id', 'yearBuilt', 'assessorID', 'taxAssessments', 
                                 'propertyTaxes', 'legalDescription', 'subdivision', 'ownerOccupied']]
                              

property_fact.drop_duplicates(subset=['id', 'location_id', 'features_id', 'owner_id'], keep='last', inplace=True)
property_fact = property_fact[property_fact['location_id'].notnull()].copy()

In [29]:
property_fact['ownerOccupied'] = property_fact['ownerOccupied'].apply(lambda x: bool(x) if pd.notnull(x) else None)

In [21]:
location_dim = location_dim.applymap(lambda x: None if pd.isna(x) else x)
features_dim = features_dim.applymap(lambda x: None if pd.isna(x) else x)
owner_dim = owner_dim.applymap(lambda x: None if pd.isna(x) else x)
property_fact = property_fact.applymap(lambda x: None if pd.isna(x) else x)

C:\Users\luk\AppData\Local\Temp\ipykernel_20384\3336098060.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  location_dim = location_dim.applymap(lambda x: None if pd.isna(x) else x)
C:\Users\luk\AppData\Local\Temp\ipykernel_20384\3336098060.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  features_dim = features_dim.applymap(lambda x: None if pd.isna(x) else x)
C:\Users\luk\AppData\Local\Temp\ipykernel_20384\3336098060.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  owner_dim = owner_dim.applymap(lambda x: None if pd.isna(x) else x)
C:\Users\luk\AppData\Local\Temp\ipykernel_20384\3336098060.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  property_fact = property_fact.applymap(lambda x: None if pd.isna(x) else x)


In [22]:
# Quite a lot of mismatched rows. Storing them here to inspect if they pop up again.

rejected_facts = property_fact[
    property_fact['location_id'].isna() | 
    property_fact['features_id'].isna() | 
    property_fact['owner_id'].isna()
].copy()


In [23]:
location_dim.to_csv('location_dim.csv', index=False)
features_dim.to_csv('features_dim.csv', index=False)
owner_dim.to_csv('owner_dim.csv', index=False)
property_fact.to_csv('property_fact.csv', index=False)

In [24]:
def get_db_connection():
    connection = psycopg2.connect(
        host = 'localhost',
        database = 'Zuma_Agency',
        user = 'postgres',
        password = 'MongoDB4luv'
    )
    return connection

In [30]:
conn = get_db_connection()
cursor = conn.cursor()

for _,row in location_dim.iterrows():
    cursor.execute('''
                        INSERT INTO zuma.location_dim (formattedAddress, addressLine1, addressLine2, 
                                                              county, zipCode, city, state, latitude, longitude)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)''',
                   (row['formattedAddress'], row['addressLine1'], row['addressLine2'], row['county'], row['zipCode'], row['city'], 
                    row['state'], row['latitude'], row['longitude'])
        )

for _,row in features_dim.iterrows():
    cursor.execute('''
                        INSERT INTO zuma.features_dim (propertyType, bedrooms, bathrooms, squareFootage, features, lotSize, hoa)
                        VALUES (%s, %s, %s, %s, %s, %s, %s)''',
                   (row['propertyType'], row['bedrooms'], row['bathrooms'], row['squareFootage'], row['features'], 
                    row['lotSize'], row['hoa'])
        )
    
for _,row in owner_dim.iterrows():
    cursor.execute('''
                        INSERT INTO zuma.owner_dim (owner)
                        VALUES (%s)''',
                   (row['owner'],)
        )
    
for _, row in property_fact.iterrows():
    cursor.execute('''
                        INSERT INTO zuma.property_fact (id, yearBuilt, assessorID, taxAssessments,
                                                   propertyTaxes, legalDescription, subdivision, ownerOccupied)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                        ON CONFLICT (id) DO NOTHING''', 
                    (row['id'], row['yearBuilt'], row['assessorID'], row['taxAssessments'],
                     row['propertyTaxes'], row['legalDescription'], row['subdivision'], row['ownerOccupied'])
        )


conn.commit()
cursor.close()
conn.close()